In [3]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import datetime as dt
import requests
import os
import urllib.parse as url_ready

# Madrid

In [3]:
madrid_path = './stock_symbols/madrid_html'

In [4]:
madrid_dir = os.listdir(madrid_path)

In [5]:
def set_column_names(df):
    df_ = df.copy()
    column_names = list(df_.ix[0])
    df_.columns = column_names
    df_.drop(labels=0, axis=0, inplace=True)
    df_.reset_index(drop=True, inplace=True)
    return df_

In [ ]:
# from the list on the website, pull out the stock names

In [6]:
df = pd.DataFrame()
for file in madrid_dir:
    with open(os.path.join(madrid_path, file), 'r') as f:
        file_text = f.read()
    file_bs = bs(file_text, 'lxml')
    table_ = file_bs.find_all(id='ctl00_Contenido_tblEmisoras')[0]
    df_temp = pd.read_html(str(table_))[0]
    df_temp = set_column_names(df_temp)
    df = df.append(df_temp)
df.reset_index(drop=True, inplace=True)

In [ ]:
# from the list of stock names, pull the tickers from google finance

In [7]:
list_stocks = list()
for _, stock_name in df['Name'].iteritems():
    resp = requests.get('https://www.google.com/finance?q={0}'.format(url_ready.quote(stock_name)))
    resp_split = resp.text.split('(BME:')
    if 'no matches' in resp.text:
        list_stocks.append({'stock_name': stock_name, 'ticker': 'none found'})
    elif len(resp_split) == 1:
        list_stocks.append({'stock_name': stock_name, 'ticker': 'more than one'})
    else:
        list_stocks.append({'stock_name': stock_name, 'ticker': 'BME:' + resp_split[1].split(')')[0]})

In [16]:
pd.DataFrame(list_stocks).to_csv(os.path.join(*(madrid_path.split('/')[:-1]), 'tickers', 'madrid_tickers.csv'))

# Milan

# Toronto

In [ ]:
# pull tickers out of html

In [37]:
toronto_exchange_code = 'TSE'
file_path = './stock_symbols/toronto_html'

In [28]:
def convert_html_header(df):
    df_ = df.copy()
    df_.columns = list(df_.loc[0])
    df_.drop(labels=0, axis=0, inplace=True)
    return df_

In [38]:
files = os.listdir(file_path)

In [51]:
df_full = pd.DataFrame()
for file in files:
    with open(os.path.join(file_path, file), 'r') as f:
        raw_html = f.read()

    toronto_bs = bs(raw_html, 'lxml')

    data_section = toronto_bs.findAll("div", { "class" : "fullTableWrapper" })
    data_section = str(data_section[0])
    df_list = pd.read_html(str(data_section))
    df = df_list[-1]
    df = convert_html_header(df)

    df['ticker'] = toronto_exchange_code + ':' + df['Symbol']
    df_full = df_full.append(df)

In [54]:
df_full = df_full[['Company Name', 'ticker']]
df_full.columns = ['stock_name', 'ticker']

In [58]:
df_full.to_csv('./stock_symbols/tickers/toronto_tickers.csv')